In [ ]:
!pip install osmnx pandas geopandas

In [ ]:
import io
import zipfile
from pathlib import Path
from datetime import date

import pandas as pd
import requests

# =========================
# Настройки и константы
# =========================
BASE = "https://download.geonames.org/export/dump"
OUT_DIR = Path("data/geonames_cities")
OUT_DIR.mkdir(parents=True, exist_ok=True)
TIMEOUT = 120

# Колонки основной таблицы 'geoname' (см. Readme GeoNames)
GEONAME_COLS = [
    "geonameid","name","asciiname","alternatenames","latitude","longitude",
    "feature_class","feature_code","country_code","cc2","admin1_code","admin2_code",
    "admin3_code","admin4_code","population","elevation","dem","timezone","modification_date"
]

# Фильтр «чистых городов»
CITY_FCODES = {"PPL","PPLC","PPLA","PPLA2","PPLA3","PPLA4","PPLG","PPLL","PPLS"}

# =========================
# Функции
# =========================
def load_iso2_country_table() -> pd.DataFrame:
    """
    Загружает countryInfo.txt и возвращает DataFrame:
    ISO — код страны (ISO2)
    Country — полное название страны
    """
    url = f"{BASE}/countryInfo.txt"
    txt = requests.get(url, timeout=TIMEOUT).text
    clean = "\n".join(l for l in txt.splitlines() if not l.startswith("#"))
    df = pd.read_csv(io.StringIO(clean), sep="\t", dtype=str, header=0)
    # По позициям (0 — ISO, 4 — Country)
    df = df.rename(columns={df.columns[0]: "ISO", df.columns[4]: "Country"})
    df = df[["ISO", "Country"]].dropna().reset_index(drop=True)
    df["ISO"] = df["ISO"].str.upper()
    return df

def safe_filename(name: str) -> str:
    """
    Очищает имя файла от запрещённых символов.
    """
    return "".join(c for c in name if c.isalnum() or c in (" ", "-", "_")).strip().replace(" ", "_")

def process_country(iso2: str, country_name: str) -> pd.DataFrame:
    """
    Скачивает {ISO2}.zip, фильтрует города, сохраняет CSV с названием страны.
    """
    url = f"{BASE}/{iso2}.zip"
    print(f"[{country_name}] downloading {url}")
    r = requests.get(url, timeout=TIMEOUT)
    if r.status_code != 200:
        print(f"[{country_name}] skip: HTTP {r.status_code}")
        return pd.DataFrame(columns=["name","population","date"])

    with zipfile.ZipFile(io.BytesIO(r.content)) as zf:
        txt_name = f"{iso2}.txt"
        if txt_name not in zf.namelist():
            print(f"[{country_name}] skip: {txt_name} not in archive")
            return pd.DataFrame(columns=["name","population","date"])

        df = pd.read_csv(
            zf.open(txt_name),
            sep="\t",
            header=None,
            names=GEONAME_COLS,
            dtype={"population": "Int64"},
            low_memory=False,
        )

    # Фильтрация
    df = df[(df["feature_class"] == "P") & (df["feature_code"].isin(CITY_FCODES))]
    df = df[df["population"].notna() & (df["population"] > 0)]

    if df.empty:
        print(f"[{country_name}] no matching cities")
        return pd.DataFrame(columns=["name","population","date"])

    out = df.loc[:, ["name", "population", "modification_date"]].rename(
        columns={"modification_date": "date"}
    )
    out["date"] = out["date"].fillna(date.today().isoformat())

    # Имя файла — по названию страны
    fname = safe_filename(country_name) + ".csv"
    path = OUT_DIR / fname
    out.to_csv(path, index=False, header=False)
    print(f"[{country_name}] saved {len(out)} rows -> {path}")
    return out

# =========================
# Точка входа
# =========================
if __name__ == "__main__":
    countries = load_iso2_country_table()
    all_parts = []

    for _, row in countries.iterrows():
        iso = row["ISO"]
        cname = row["Country"]
        try:
            part = process_country(iso, cname)
            if not part.empty:
                tmp = part.copy()
                tmp["country"] = cname
                all_parts.append(tmp)
        except Exception as e:
            print(f"[{cname}] error: {e}")

    if all_parts:
        all_df = pd.concat(all_parts, ignore_index=True)
        all_path = OUT_DIR / "ALL_cities.csv"
        all_df.to_csv(all_path, index=False, header=False)
        print(f"[ALL] total {len(all_df)} rows -> {all_path}")


In [ ]:
import os
import io
import time
import pickle
import unicodedata
from pathlib import Path

import osmnx as ox
import pandas as pd
import geopandas as gpd

from blocksnet.blocks.cutting import preprocess_urban_objects, cut_urban_blocks  # type: ignore
from blocksnet.blocks.assignment import assign_land_use                          # type: ignore
from blocksnet.enums import LandUse                                              # type: ignore

# ---------- Настройки ----------
INPUT_DIR  = Path("data/geonames_cities")   # где лежат CSV "Country.csv"
OUTPUT_DIR = Path("data/blocks")            # сюда будут pkl в подпапки по странам
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Фильтр по населению
POP_MIN = 100_000
POP_MAX = 100_000_000

# OSMnx: вежливые настройки
ox.settings.default_user_agent = "blocksnet-city-pipeline/1.0 (contact: you@example.com)"
ox.settings.nominatim_pause = 1.0
ox.settings.overpass_rate_limit = True
ox.settings.timeout = 180

CRS_OUT = "EPSG:3857"

# ---------- Справочники ----------
RULES = {
    'commercial': LandUse.BUSINESS,
    'industrial': LandUse.INDUSTRIAL,
    'cemetery': LandUse.SPECIAL,
    'garages': LandUse.INDUSTRIAL,
    'residential': LandUse.RESIDENTIAL,
    'retail': LandUse.BUSINESS,
    'grass': LandUse.RECREATION,
    'farmland': LandUse.AGRICULTURE,
    'construction': LandUse.SPECIAL,
    'brownfield': LandUse.INDUSTRIAL,
    'forest': LandUse.RECREATION,
    'recreation_ground': LandUse.RECREATION,
    'religious': LandUse.SPECIAL,
    'flowerbed': LandUse.RECREATION,
    'military': LandUse.SPECIAL,
    'landfill': LandUse.TRANSPORT
}

BC_TAGS = {
    'roads': {
        "highway": [
            "construction", "crossing", "living_street", "motorway", "motorway_link", "motorway_junction",
            "pedestrian", "primary", "primary_link", "raceway", "residential", "road", "secondary",
            "secondary_link", "services", "tertiary", "tertiary_link", "track", "trunk", "trunk_link",
            "turning_circle", "turning_loop", "unclassified"
        ],
        "service": ["living_street", "emergency_access"]
    },
    'railways': {"railway": "rail"},
    'water': {
        'riverbank': True, 'reservoir': True, 'basin': True, 'dock': True,
        'canal': True, 'pond': True, 'natural': ['water', 'bay'],
        'waterway': ['river', 'canal', 'ditch'], 'landuse': 'basin', 'water': 'lake'
    }
}

# ---------- Утилиты ----------
def safe_filename(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    s = "".join(c for c in s if c.isalnum() or c in (" ", "-", "_", ","))
    return s.strip().replace(" ", "_")

def geocode_city(city: str, country: str) -> gpd.GeoDataFrame:
    # Структурный запрос точнее
    try:
        gdf = ox.geocode_to_gdf({"city": city, "country": country})
        if gdf is not None and not gdf.empty and gdf.geometry.iloc[0].geom_type in ("Polygon", "MultiPolygon"):
            return gdf
    except Exception:
        pass
    label = f"{city}, {country}"
    for which in (None, 1, 2, 3, 4):
        try:
            gdf = ox.geocode_to_gdf(label, which_result=which)
            if gdf is not None and not gdf.empty:
                return gdf
        except Exception:
            time.sleep(1)
            continue
    raise RuntimeError(f"Не найдены границы: {city}, {country}")

# ---------- Основная функция обработки ----------
def process_and_save_city(city: str, country: str, out_country_dir: Path, crs=CRS_OUT) -> gpd.GeoDataFrame:
    out_country_dir.mkdir(parents=True, exist_ok=True)

    boundaries = geocode_city(city, country).to_crs(crs)
    poly_4326 = boundaries.to_crs(4326).union_all()
    poly_out  = boundaries.union_all()

    # water
    try:
        water = ox.features_from_polygon(poly_4326, BC_TAGS['water']).reset_index(drop=True)
        water = water[water.geom_type.isin(['Polygon','MultiPolygon','LineString','MultiLineString'])].copy()
        water = None if water.empty else water.to_crs(crs)
    except Exception:
        water = None

    # roads
    try:
        roads = ox.features_from_polygon(poly_4326, BC_TAGS['roads']).reset_index(drop=True)
        roads = roads[roads.geom_type.isin(['LineString','MultiLineString'])].copy()
        roads = None if roads.empty else roads.to_crs(crs)
    except Exception:
        roads = None

    # railways
    try:
        railways = ox.features_from_polygon(poly_4326, BC_TAGS['railways']).reset_index(drop=True)
        railways = railways[railways.geom_type.isin(['LineString','MultiLineString'])].copy()
        railways = None if railways.empty else railways.to_crs(crs)
    except Exception:
        railways = None

    lines, polygons = preprocess_urban_objects(
        roads_gdf=roads, railways_gdf=railways, water_gdf=water
    )
    blocks = cut_urban_blocks(boundaries, lines, polygons)

    # landuse
    try:
        fz = ox.features_from_polygon(poly_4326, tags={'landuse': True})
        fz = fz[fz.geom_type.isin(['Polygon', 'MultiPolygon'])].copy()
        fz = None if fz.empty else fz.reset_index(drop=True)[['geometry','landuse']].rename(columns={'landuse':'functional_zone'}).to_crs(crs)
    except Exception:
        fz = None

    if fz is not None:
        blocks = assign_land_use(blocks, fz.reset_index(drop=True), RULES)
        blocks.dropna(subset=['land_use'], inplace=True)
        blocks = blocks[['geometry', 'land_use', 'share']].reset_index(drop=True)
    else:
        blocks['land_use'] = None
        blocks['share'] = None

    # save
    fname = f"{city}.pkl"
    with open(out_country_dir / fname, "wb") as f:
        pickle.dump(blocks, f)
    return blocks

# ---------- Запуск по всем CSV со странами ----------
def run_all():
    csv_files = sorted(INPUT_DIR.glob("*.csv"))  # файлы типа Country.csv
    for csv_path in csv_files:
        country = csv_path.stem.replace("_", " ")
        out_country_dir = OUTPUT_DIR / safe_filename(country)

        print(f"\n=== {country} ===")
        try:
            df = pd.read_csv(csv_path, header=None, names=['name','population','date'])
        except Exception as e:
            print(f"[{country}] не удалось прочитать CSV: {e}")
            continue

        # фильтр населения
        try:
            df['population'] = df['population'].astype(int)
        except Exception:
            df = df[pd.to_numeric(df['population'], errors='coerce').notna()]
            df['population'] = df['population'].astype(int)

        df = (df.drop_duplicates(subset=['name'])
                .query(f"{POP_MIN} <= population <= {POP_MAX}")
                .sort_values('population', ascending=False))

        if df.empty:
            print(f"[{country}] нет городов в диапазоне {POP_MIN}-{POP_MAX}")
            continue

        for city in df['name'].tolist():
            label = f"{city}, {country}"
            print(f" -> {label}")
            try:
                process_and_save_city(city, country, out_country_dir)
            except Exception as e:
                print(f"    [skip] {label}: {e}")
                continue
            # бережём Nominatim/Overpass
            time.sleep(1.0)

if __name__ == "__main__":
    run_all()


d:\programming\github\loading-cities-for-land-use\.venv\lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 16 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
2025-08-15 17:37:04.608 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:44 - Checking roads schema
2025-08-15 17:37:04.634 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:51 - Checking railways schema
2025-08-15 17:37:04.645 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:58 - Checking water schema
2025-08-15 17:37:04.666 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:21 - Checking boundaries schema
2025-08-15 17:37:04.677 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:25 - Checking line objects schema
2025-0

 -> Ballarat, Australia
    [skip] Ballarat, Australia: At least one GeoDataFrame must be passed
 -> Bendigo, Australia


2025-08-15 17:38:39.892 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:44 - Checking roads schema
2025-08-15 17:38:39.908 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:51 - Checking railways schema
2025-08-15 17:38:39.919 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:58 - Checking water schema
2025-08-15 17:38:39.935 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:21 - Checking boundaries schema
2025-08-15 17:38:39.946 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:25 - Checking line objects schema
2025-08-15 17:38:39.962 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:31 - Checking polygon objects schema
2025-08-15 17:38:39.976 | INFO     | blocksnet.blocks.cutting.processing.core:_exclude_polygons:45 - Excluding polygon objects from blocks
2025-08-15 17:38:39.990 | INFO     | blocksnet.blocks.cutting.processing.core:_get_enclosures:51 -


=== Austria ===
 -> Vienna, Austria


2025-08-15 17:41:32.242 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:44 - Checking roads schema
2025-08-15 17:41:32.583 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:51 - Checking railways schema
2025-08-15 17:41:32.613 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:58 - Checking water schema
2025-08-15 17:41:32.734 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:21 - Checking boundaries schema
2025-08-15 17:41:32.747 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:25 - Checking line objects schema
2025-08-15 17:41:32.838 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:31 - Checking polygon objects schema
2025-08-15 17:41:32.854 | INFO     | blocksnet.blocks.cutting.processing.core:_exclude_polygons:45 - Excluding polygon objects from blocks
2025-08-15 17:41:33.275 | INFO     | blocksnet.blocks.cutting.processing.core:_get_enclosures:51 -

...

2025-08-15 20:48:16.148 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:44 - Checking roads schema
2025-08-15 20:48:16.286 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:51 - Checking railways schema
2025-08-15 20:48:16.305 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:58 - Checking water schema
2025-08-15 20:48:16.306 | WARNING  | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:60 - Water GeoDataFrame is None. Creating empty
2025-08-15 20:48:16.364 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:21 - Checking boundaries schema
2025-08-15 20:48:16.374 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:25 - Checking line objects schema
2025-08-15 20:48:16.441 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:31 - Checking polygon objects schema


    [skip] Fortaleza, Brazil: Rows count must be greater than 0
 -> Manaus, Brazil


2025-08-15 20:50:13.640 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:44 - Checking roads schema
2025-08-15 20:50:13.726 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:51 - Checking railways schema
2025-08-15 20:50:13.727 | WARNING  | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:53 - Railways GeoDataFrame is None. Creating empty
2025-08-15 20:50:13.731 | INFO     | blocksnet.blocks.cutting.preprocessing.core:preprocess_urban_objects:58 - Checking water schema
2025-08-15 20:50:13.768 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:21 - Checking boundaries schema
2025-08-15 20:50:13.777 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:25 - Checking line objects schema
2025-08-15 20:50:13.940 | INFO     | blocksnet.blocks.cutting.processing.core:wrapper:31 - Checking polygon objects schema
2025-08-15 20:50:13.953 | INFO     | blocksnet.blocks.cutting.processing.core:_e

 -> Brasília, Brazil


d:\programming\github\loading-cities-for-land-use\.venv\lib\site-packages\osmnx\_overpass.py:271: UserWarning: This area is 6,256 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)
